# Unit 3 Lesson 4 Project 3: Profiling and Testing Code

#### Estimated time 30 minutes

## Write Unit Tests

Before learning to test code, you should first understand a bit about code quality. As you continue your journey into programming, you're going to write a lot of code. At least one of your projects is going to grow into a big hairy mess of files and functions - it's inevitable. When you go to write a new function on top of this huge codebase, you'll want to make sure that adding new code doesn't break any old code. So how should you proceed? By writing test cases.

Sometimes called "unit testing" or "test-driven development" (TDD), writing tests for your code is standard nowadays. The basic idea is this: every piece of new code must have a test associated with it, and every test must pass before pushing any code to the git repo.

That is quite abstract and vague, so here's a simple example. Let's write a test for fizzbuzz, and then implement a fizzbuzz function that passes the test:

In [1]:
def fizzbuzz_test(f):
    if f(3) == "fizz" and f(5) == "buzz" and f(15) == "fizzbuzz":
        print("Success!")
    else:
        print("Nope. Try again.")

The function fizzbuzz_test() takes a function as an argument, runs it through a few tests, and then tells you if the function passed or not.

Try the test function with the following fizzbuzz implementation:

In [2]:
def fizzbuzz(n):
    ret = ""
    if not (n%3):
        ret += "fizz"
    if not (n%5):
        ret += "buzz"
    return ret or str(n)

If you run the test, it passes successfully:

In [3]:
fizzbuzz_test(fizzbuzz)

Success!


However, you'll notice that you are only testing 3 points: f(3), f(5), and f(15). You could totally write a fizzbuzz implementation like this:

In [5]:
def fizzjoke(n):
    if n == 3:
        return "fizz"
    if n == 5:
        return "buzz"
    if n == 15:
        return "fizzbuzz"

In [6]:
fizzbuzz_test(fizzjoke)

Success!


...which still passes the test, but it doesn't actually implement fizzbuzz. A better fizzbuzz test would look like this:

In [7]:
def fizzbuzz_goodtest(f):
    output = []
    for n in range(100):
        output.append(str(f(n) + "\n"))

    expected = open("fizzbuzz-output.txt", "r")
    i = 0
    for line in expected:
        if line == output[i]:
            print("Success!")
            i += 1
        else:
            print("Nope. Try Again.")

The file "fizzbuzz-output.txt" contains 100 lines of expected output. `fizzbuzz_goodtest()` simply compares this file to 100 executions of fizzbuzz. If it passes, it prints "Success!". Running `fizzbuzz_goodtest(fizzbuzz)` will print "Success!" to the terminal 100 times.

This is all well and good, but when testing more complicated code, you will find yourself writing lots of boilerplate code. Naturally, as programmers, we seek to automate things, and so Python ships with a module called unittest that you can use to write more concise tests.

The `unittest` module provides a standardized class for writing test cases, as well as functions for setting up and tearing down a test environment. For example imagine you needed to test some database transactions. `unittest` would give you tools for automating setting up the database, populating it with test data, and tearing it down after the tests are complete.

To get started, we'll show you the general form of setting up unit tests. Create a new file called `example_tests.py`. In that file, add a stub for your test class:

In [10]:
import unittest

class ExampleTests(unittest.TestCase):
    pass

if __name__ == "__main__":
    unittest.main()

E
ERROR: /run/user/1000/jupyter/kernel-2815449b-ea98-4bdf-9f66-0d1fbd31714d (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/run/user/1000/jupyter/kernel-2815449b-ea98-4bdf-9f66-0d1fbd31714d'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

To exit: use 'exit', 'quit', or Ctrl-D.


Here you import the unittest module, and create a new subclass of unittest.TestCase called ExampleTests. In your main block you call the unittest.main function, which will collect and run any tests contained in the file.

Save the file, then run it with python example_tests.py. You should see the following output:

`---`
`Ran 0 tests in 0.000s`

`OK`

This tells you that you didn't run any tests (which is unsurprising, since you haven't written any yet), and that there were no test failures. To build this out, copy `fizzbuzz_goodtest()` from above and paste it in place of `pass`. Copy `fizzbuzz()` and place it somewhere else in the file, but not inside the `ExampleTests` class. In order to run the unit tests, you have to have the code you are testing available to the test cases. Usually, you would `import` the code that you're testing, but for this simple example we'll just copy and paste the code.

Now, run the `example_tests.py` file again, and you'll see that the unittest module automatically ran the test for you and reported the success/failure rate.

## Profiling your code

When your code is running too slowly, it's time to figure out why. Usually this is due to a bottleneck in the code where the processing slows down. You can identify these bottlenecks by profile your code.

### Note

While profiling isn't a necessary task, it's important when you begin writing code that's going into a production environment where issues of scale and reliability become important. Getting into these good habits and best practices now will help you immensely in your career if you choose to be a professional data scientist.

The following section is adapted from this wonderful StackOverflow answer [http://stackoverflow.com/questions/3045556/how-to-profile-my-code/9594242#9594242]:

### Profiling with cProfile

cProfile [https://docs.python.org/2/library/profile.html#instant-user-s-manual] is one of the most used python profilers out there, and although very powerful, the standard text output is somewhat lackluster. Here I'll show you how to use cProfile on your application in an easier way.

There are two common ways to use cProfile, you can use it as a command in prompt to profile a given module, or you can use it inside your code, to profile specific snippets of code.

#### Profiling a module

To use cProfile to profile an entire module, simply use the following command in your prompt:

`python -m cProfile -o output_filename.pstats path/to/script arg1 arg2`

This will run your module with the given arguments (they are optional) and dump the output in `output_filename.pstats`.

There are lots of ways [https://docs.python.org/2/library/profile.html] to read the data on that output file, but for the purpose of this post, let's not worry about those and focus on getting that graphical visualization.

#### Profiling from inside

Sometimes you don't want to profile an entire module, just a few lines of it.

To do so, you are gonna have to add some code to your module.

First of all:

`import cprofile`

And then, you can replace any segment of code with the following:

`cProfile.runctx('Your code here', globals(), locals(), 'output_file')`

For example, here is a test before and after profiling:

In [11]:
import unittest

class Test(unittest.TestCase):

    def testSomething(self):
        self.DoSomethingIDontCareAbout()

        param = 'whatever'
        self.RunFunctionIThinkIsSlow(param)

        self.AssertSomeStuff() # This is after all, a test

After:

In [12]:
import unittest
import cProfile

class Test(unittest.TestCase):

    def testSomething(self):
        self.DoSomethingIDontCareAbout()

        param = 'whatever'
        cProfile.runctx(
            'self.RunFunctionIThinkIsSlow(param)',
            globals(),
            locals(),
            'myProfilingFile.pstats'
        )

        self.AssertSomeStuff() # This is after all, a test

Converting a pstats file to a graph

To convert your profiling file to a graph, you will need a couple of things:

    gprof2dot [https://github.com/jrfonseca/]: This module will convert your output into a dot file, a standard file format for graph descriptions.
    GraphViz [www.graphviz.org/Download.php]: It turns your dot file into an image.

After you have downloaded gprof2dot and installed GraphViz, run this command in your prompt:

`python gprof2dot -f pstats myProfileFile | dot -Tpng -o image_output.png`

You might have to use a complete path for gprof2dot and/or dot, or you could add them to your PATH environment variable.

After all of this, you should have an image that looks like this:

![results example][http://i.stack.imgur.com/JKcPb.png]

* Hotter colors (red, orange, yellow) indicate functions that take up more of the total runtime than colder colors (green, blue)

* On each node, you can see what percentage of the total runtime that function used and how many times it was called.

* Arrows between nodes indicate which function called other functions, and such arrows also have a caption indicating what percentage of the runtime came through there.

Note: percentages won't always add up to 100%, especially on code sections that reference C++ code, which won't be profiled. cProfile also won't be able to determine what's called from inside an eval statement, so you might see some jumps in your graph.

## Challenge

* (Optional) If you feel confident writing unit tests to your code, try it out and see. This is best to try only if you have a lot of practice writing code, otherwise feel free to move on.

## Submission

Commit your refactored code with comments on the changes you made using the link below.